# <center>Linear Model season error

In [67]:
%run C:/Users/knightm-adm/ML/Level_1_Samples/Select_sales.ipynb
path = "C:/Users/knightm-adm/ML/Level_1_Samples/Sample_span90_minmax8_training_100_target_100"
files = glob.glob(path + '*', recursive=False)
error_data = pd.concat((pd.read_csv(i) for i in files)).reset_index(drop = True) 

Total sales: 13303732 with Total Customers: 37291
Training starts on 19th December 2018 for 300 business days until 01th March 2020 
Target/actual starts on 02th March 2020 for 250 business days until 01th March 2021
Total sales: 4639145 with Total Customers: 26856
Total sales: 4525240 with Total Customers: 14239
Total sales: 3704804 with Total Customers: 6970
Total sales: 1075613 with Total Customers: 2000


In [72]:
path = "C:/Users/knightm-adm/ML/Level_1_Samples/Sample_span90_minmax8_training_200_target_100"
files = glob.glob(path + '*', recursive=False)
error_data = pd.concat((pd.read_csv(i) for i in files)).reset_index(drop = True) 
v = pd.unique(error_data['Customer'])
len(v)

2000

In [66]:
error_data.shape

(12632799, 20)

In [69]:
v = pd.unique(error_data['Customer'])
len(v)

10045

In [14]:
path = "C:/Users/knightm-adm/ML/Level_1_Samples/Season_"
files = glob.glob(path + '*', recursive=False)
season_data = pd.concat((pd.read_csv(i) for i in files)).reset_index(drop = True) 

In [40]:
path = "C:/Users/knightm-adm/ML/Level_1_Samples/Target_"
files = glob.glob(path + '*', recursive=False)
all_target_sales = pd.concat((pd.read_csv(i) for i in files)).reset_index(drop = True) 

In [41]:
import pandas as pd; import numpy as np
import matplotlib.pyplot as plt
import cx_Oracle
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None
from sqlalchemy import create_engine
import datetime
from datetime import timedelta
import random
import glob
import math
import os

In [42]:
season_data.sample(14)

,Month_year,PC_spend
0,Jan_2019,0.09
34,Nov_2021,0.10
20,Sep_2020,0.09
24,Jan_2021,0.07
10,Nov_2019,0.11
26,Mar_2021,0.08
12,Jan_2020,0.09
4,May_2019,0.08
3,Apr_2019,0.08
8,Sep_2019,0.08


In [43]:
training_sample = error_data
training_sample.tail(1)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,Deleted,PRODUCTID,QUOTATIONLINEID,Bus_Type,span%,training_range,target_range,Month,Month_year,Year
12632798,39688785,926,64971,2020-11-19,N,1,3.88,Y,146943996,NaN,NaN,263955,0.00,54,1.00,300,50,11,Nov_2020,2020


In [44]:
training_sample['Created'] = pd.to_datetime(training_sample['Created'])
d = {1: 'Jan_', 2:'Feb_', 3:'Mar_',
     4:'Apr_', 5:'May_', 6:'Jun_',
      7:'Jul_', 8:'Aug_', 9:'Sep_',
     10:'Oct_', 11:'Nov_', 12:'Dec_'}
training_sample['Month'] = training_sample['Created'].dt.month
training_sample['Month_year'] = training_sample['Month'].map(d)
training_sample['Year'] = training_sample['Created'].dt.year
training_sample['Month_year'] = (training_sample['Month_year'] + training_sample['Year'].astype(str) )
training_sample.tail(1)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,Deleted,PRODUCTID,QUOTATIONLINEID,Bus_Type,span%,training_range,target_range,Month,Month_year,Year
12632798,39688785,926,64971,2020-11-19,N,1,3.88,Y,146943996,NaN,NaN,263955,0.00,54,1.00,300,50,11,Nov_2020,2020


In [45]:
all_target_sales['Created'] = pd.to_datetime(all_target_sales['Created'])
d = {1: 'Jan_', 2:'Feb_', 3:'Mar_',
     4:'Apr_', 5:'May_', 6:'Jun_',
      7:'Jul_', 8:'Aug_', 9:'Sep_',
     10:'Oct_', 11:'Nov_', 12:'Dec_'}
all_target_sales['Month'] = all_target_sales['Created'].dt.month
all_target_sales['Month_year'] = all_target_sales['Month'].map(d)
all_target_sales['Year'] = all_target_sales['Created'].dt.year
all_target_sales['Month_year'] = (all_target_sales['Month_year'] + all_target_sales['Year'].astype(str) )
all_target_sales.tail(1)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,Deleted,PRODUCTID,QUOTATIONLINEID,Bus_Type,Month,Month_year,Year
8561525,40645341,926,188728,2021-01-27,N,1,72.00,Y,150310941,NaN,NaN,1619700,0.00,54,1,Jan_2021,2021


In [46]:
training_sample = pd.merge(training_sample, season_data, left_on="Month_year", 
                               right_on="Month_year", how="left")
training_sample['Season_adj'] = (1/12) / training_sample['PC_spend'] 
training_sample['Season_cost'] = training_sample['Cost'] * training_sample['Season_adj']
training_sample.head(4)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,...,Bus_Type,span%,training_range,target_range,Month,Month_year,Year,PC_spend,Season_adj,Season_cost
0,37027587,101,92461,2020-05-19,N,1,9.04,Y,137221804,NaN,...,54,0.98,100,100,5,May_2020,2020,0.06,1.41,12.73
1,37323856,101,74222,2020-06-15,N,1,11.80,Y,138313764,NaN,...,54,0.96,100,100,6,Jun_2020,2020,0.08,1.05,12.44
2,37323856,101,74222,2020-06-15,N,1,20.92,Y,138313765,NaN,...,54,0.96,100,100,6,Jun_2020,2020,0.08,1.05,22.05
3,37323856,101,74222,2020-06-15,N,1,8.74,Y,138313766,NaN,...,54,0.96,100,100,6,Jun_2020,2020,0.08,1.05,9.21


In [47]:
all_target_sales = pd.merge(all_target_sales, season_data, left_on="Month_year", 
                               right_on="Month_year", how="left")
all_target_sales['Season_adj'] = (1/12) / all_target_sales['PC_spend'] 
all_target_sales['Season_cost'] = all_target_sales['Cost'] * all_target_sales['Season_adj']
all_target_sales.sample(4)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,Deleted,PRODUCTID,QUOTATIONLINEID,Bus_Type,Month,Month_year,Year,PC_spend,Season_adj,Season_cost
4355048,39876740,304,79744,2020-11-30,N,1,2.99,Y,147612164,NaN,NaN,1614084,0.00,54,11,Nov_2020,2020,0.09,0.90,2.70
2981475,39040698,160,38662,2020-10-09,N,1,8.25,Y,144626787,NaN,NaN,461708,0.00,54,10,Oct_2020,2020,0.10,0.83,6.85
1600140,38164508,1409,94845,2020-08-12,N,1,2.21,Y,141437081,NaN,NaN,714691,0.00,199,8,Aug_2020,2020,0.08,1.11,2.45
1394868,41081977,646,180251,2021-02-25,N,1,8.64,Y,151892199,NaN,NaN,139894,0.00,54,2,Feb_2021,2021,0.08,1.10,9.47


In [48]:
def find_trading_days_count(all_sales):
    total_days = all_sales[['Customer','Created']]
    total_days = total_days.drop_duplicates()
    total_days = pd.DataFrame(total_days['Created'].groupby([total_days['Customer']]).count())
    total_days.rename(columns = {'Created':'Total purchase days'}, inplace = True)
    total_days = total_days.reset_index(col_level=0)
    return total_days['Total purchase days'][0]

In [49]:
def find_trading_df(c_data, training_start, training_period, holidays, c):
    trading_df = pd.DataFrame(0 for i in range(len(training_period)))
    trading_df.rename(columns = {0:'Date'}, inplace = True)
    trading_df['Date'] = pd.Series(pd.bdate_range(training_start, periods=len(training_period), holidays=holidays, freq='C').normalize())
    trading_df['Date'] =  pd.to_datetime(trading_df['Date'])
    trading_df['Customer'] = c
    c_trading_df = c_data
    c_trading_df = pd.DataFrame(c_data[['Cost']].groupby([c_data['Created']]).sum())
    c_trading_df = c_trading_df.reset_index(col_level=0)
    c_trading_df.rename(columns = {'Cost':'Total spend'}, inplace = True)
    c_trading_df['Created'] = pd.to_datetime(c_trading_df['Created'])
    c_trading_df_sat = c_trading_df[~c_trading_df['Created'].isin(trading_df['Date'])]
   
    trading_df = pd.merge(trading_df, c_trading_df, left_on="Date", 
                               right_on="Created", how="left")
    
    trading_df = pd.merge_asof(trading_df, c_trading_df_sat, left_on="Date", 
                               right_on="Created", tolerance=pd.Timedelta("1d"), direction='forward')
    c_trading_df_sun = c_trading_df_sat[~c_trading_df_sat['Created'].isin(trading_df['Created_y'])]
    trading_df = pd.merge_asof(trading_df, c_trading_df_sun, left_on="Date", 
                               right_on="Created", tolerance=pd.Timedelta("1d"), direction='backward')
    trading_df['Spend'] = trading_df.fillna(0)['Total spend_x'] + trading_df.fillna(0)['Total spend_y'] + trading_df.fillna(0)['Total spend']
    trading_df = trading_df[['Date', 'Customer', 'Spend']]
    return trading_df

In [50]:
def find_bins(trading_df, training_period):
    trading_days = len(training_period)
    trading_df['all_X'] = np.arange(0, trading_days, 1, dtype=None)
    td = int(trading_days / 10)
    bins_config = list(range(-1, trading_days, td))
    bins = pd.cut(trading_df['all_X'], bins=bins_config)
    labels = [0,1,2,3,4,5,6,7,8,9]
    trading_df['binned'] = pd.cut(trading_df['all_X'], bins_config, labels=labels)  
    LM_bins = pd.DataFrame(trading_df.groupby(bins)['Spend'].agg(['count','sum'])).reset_index()
    LM_bins['model_X'] = np.arange(0, 10, 1, dtype=None)
    return LM_bins

In [ ]:
LM_bins 

In [51]:
all_target_sales.sample(2)

,ID,Branch,Customer,Created,Cancelled,Type,Cost,Fulfilled,LineID,SO_LineID,Deleted,PRODUCTID,QUOTATIONLINEID,Bus_Type,Month,Month_year,Year,PC_spend,Season_adj,Season_cost
52226,40297427,1289,189729,2021-01-04,N,1,19.00,Y,149054654,NaN,NaN,735210,0.00,248,1,Jan_2021,2021,0.07,1.13,21.51
1215324,40750396,367,57375,2021-02-03,N,1,4.55,Y,150692728,NaN,NaN,1135933,0.00,261,2,Feb_2021,2021,0.08,1.10,4.99


In [58]:
training_sample.shape

(12632799, 23)

In [64]:
v = pd.unique(error_data['Customer'])
len(v)

10045

###### methods for season adjusted model

In [53]:
Customers = []

Purchases = []
Purchase_days = []
Spend = []
unadjPredicted = []
Predicted = []
predicted = []
Actual = []
MSE = []
MAE = []
Intercept = []
Coef = []
Span = []

for c in pd.unique(training_sample['Customer']):

    c_data = training_sample[ training_sample['Customer'] == c]
    c_target = all_target_sales[ all_target_sales['Customer'] == c]
    
    Customers.append(c)
    Span.append(c_data['span%'].iloc[0])
    Purchases.append(len(c_data))
    Actual.append(c_target['Cost'].sum())

    Purchase_days.append(find_trading_days_count(c_data))
    Spend.append(c_data['Cost'].sum())

    trading_df = find_trading_df(c_data, training_start, training_period, holidays, c)
    bins = find_bins(trading_df, training_period)


    X = bins[['model_X']]
    y = bins['sum'].fillna(0)
    SD = bins['sum'].std()
    lin_reg = LinearRegression()
    lin_reg.fit(X.values.reshape(-1, 1), y)
    predict_X = lin_reg.predict(X)
    predict_all_X = predict_X.sum()

    z = np.arange(10, 20, 1, dtype=None)

    predict_values = (len(target_period) / len(training_period) ) * 10

    predicted = []
    for i in z:
        predict_val = 0
        if predict_values <= 0:

            predict_values = predict_values  - 1
        elif predict_values < 1:
            if lin_reg.intercept_ + (i * predict_values * lin_reg.coef_[0]) < 0:
                0
            else:
                predict_val = lin_reg.intercept_ + (i * predict_values * lin_reg.coef_[0]) * 4
        elif predict_values >= 1:
            if lin_reg.intercept_ + (i * lin_reg.coef_[0]) < 0:
                0
            else:            
                predict_val = lin_reg.intercept_ + (i * lin_reg.coef_[0]) 
        predicted.append(np.sum(predict_val))
        predict_values = predict_values  - 1
    
    
    actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
    unadjPredicted.append(np.sum(predicted))
    Predicted.append(np.sum(predicted) * actual_season_adj  )
    
    MSE.append(mean_squared_error(y, predict_X))
    MAE.append(mean_absolute_error(y, predict_X))
    Intercept.append(lin_reg.intercept_)
    Coef.append(lin_reg.coef_[0])

    zipped = list(zip(Customers, Span, Purchases, Purchase_days, Spend, MAE, MSE, Intercept, Coef, Predicted, unadjPredicted, Actual))
    results = pd.DataFrame(zipped, columns=['Customers', 'Span','Purchases', 'Purchase_days', 'Spend', 'MAE', 'MSE', 
                                            'Intercept', 'Coef','Predicted', 'Unseasoned','Actual']) 
    results['Predicted'] = results['Predicted'].fillna(0)
    results = results.sort_values('Spend')
    results['MAE_pc'] = results['MAE'] / results['Spend'] 
    results.tail(25)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  w

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserW

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegr

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserW

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\knightm-adm\AppData\Local\Temp\5\ipykernel_8188\2163892138.py:65: RuntimeWarning: invalid value encountered in double_scalars
  actual_season_adj = c_target['Cost'].sum() / c_target['Season_cost'].sum()
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegr

,Customers,Span,Purchases,Purchase_days,Spend,MAE,MSE,Intercept,Coef,Predicted,Unseasoned,Actual,MAE_pc
8569,129008,0.96,9,8,38.50,4.67,34.12,5.05,-0.27,0.00,10.31,0.00,0.12
9729,74223,0.92,13,8,57.53,4.40,31.11,8.29,-0.56,0.00,2.40,0.00,0.08
9004,96593,0.90,13,10,68.76,6.79,79.08,-0.80,1.33,0.00,167.81,0.00,0.10
5790,98808,0.99,11,9,69.92,3.99,26.53,-3.31,1.79,249.16,206.42,204.80,0.06
9588,103404,0.94,16,11,74.25,6.51,78.04,11.05,-0.81,0.00,-1.15,0.00,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,69661,1.00,74349,594,3382687.47,31995.98,1376910660.44,72539.37,27899.70,4378646.44,4335615.15,1758018.69,0.01
79,70209,1.00,39790,405,4165876.76,26905.32,1108566714.07,-44086.35,22458.02,2480997.77,2567681.05,2587899.62,0.01
720,38573,1.00,16066,337,4289463.59,85686.40,10231737660.78,-112845.94,65615.83,7638207.04,7645676.72,7032636.38,0.02
167,34592,1.00,24601,361,4886595.20,41153.77,2419954207.40,-51773.05,31793.90,3558131.97,3730837.79,7074300.70,0.01


From this output we can create further variables for machine learning models. We want to express each customers trend as far as possible. We have variables for slope, intercept and spend, but we can add absolute error and even express that as a percentage of spend. We can also add a value for the absolute value of our coefficient. The best way to test the value of these further variables (as well as the whole model) will be in a lASSO model which will show shrinkage for variables with lower explanatory power.

In [54]:
results = results.sort_values('Spend')

In [55]:
results['Predicted'].sum() /  results['Actual'].sum()

2.1709813333581747

In [56]:
results['Error'] = results['Predicted'] - results['Actual'] 

results['ABS Error'] = abs(results['Predicted'] - results['Actual'] )
results['Error per day'] = abs(results['Predicted'] - results['Actual'] ) / len(target_period)
results['Target spend per day'] = results['Actual'] / len(target_period)
results['Actual per day'] = results['Spend'] / len(training_period)
results['MAE per day'] = results['MAE'] / int(len(training_period) / 10)
results['Model difference'] = results['MAE per day'] - results['Error per day']
results = results.sort_values('Model difference')
results

,Customers,Span,Purchases,Purchase_days,Spend,MAE,MSE,Intercept,Coef,Predicted,Unseasoned,Actual,MAE_pc,Error,ABS Error,Error per day,Target spend per day,Actual per day,MAE per day,Model difference
2558,1570,0.94,6364,298,2624739.40,94820.88,13527459421.27,-28544.30,48228.57,6382032.96,6109272.12,36367.52,0.04,6345665.44,6345665.44,25382.66,145.47,8749.13,3160.70,-22221.97
1288,24576,1.00,23802,351,2501244.41,98798.00,17512545639.09,22010.94,38563.54,5579766.64,5288486.00,241114.47,0.04,5338652.17,5338652.17,21354.61,964.46,8337.48,3293.27,-18061.34
3989,82545,0.97,14807,303,2542676.85,39895.71,2760765621.37,-25108.82,13105.29,1461736.89,1503918.76,5970490.94,0.02,-4508754.05,4508754.05,18035.02,23881.96,8475.59,1329.86,-16705.16
4409,31827,1.00,2211,113,2237133.29,182364.86,62499927168.89,-149364.27,58455.88,6497707.33,6371897.38,1103328.36,0.08,5394378.97,5394378.97,21577.52,4413.31,7457.11,6078.83,-15498.69
669,82089,0.99,90519,364,5018906.33,99120.51,16087968585.20,-140749.25,73040.22,8397419.26,8374565.65,4050800.46,0.02,4346618.80,4346618.80,17386.48,16203.20,16729.69,3304.02,-14082.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9683,125093,0.95,1480,81,260844.59,17534.96,514909635.62,42341.92,-3612.77,0.00,-35548.85,0.00,0.07,0.00,0.00,0.00,0.00,869.48,584.50,584.50
3074,125950,0.95,2450,133,407084.85,26089.92,1436932702.91,-18203.46,9232.73,981062.25,1054889.30,912705.89,0.06,68356.36,68356.36,273.43,3650.82,1356.95,869.66,596.24
9285,154054,0.91,220,40,399270.92,25018.16,1194929030.33,-25447.53,13145.32,0.00,1506154.06,0.00,0.06,0.00,0.00,0.00,0.00,1330.90,833.94,833.94
8216,154278,0.98,1994,112,325985.84,34889.82,2092822116.95,-8754.66,9189.61,0.00,1134236.58,0.00,0.11,0.00,0.00,0.00,0.00,1086.62,1162.99,1162.99


In [64]:
results2 = results.loc[(results ['Intercept'] > 0 )]
results2['Predicted'].sum(),  results2['Actual'].sum(), results2['Predicted'].sum()- results2['Actual'].sum(), results2['Predicted'].sum() / results2['Actual'].sum()

(125663500.32241932, 112044028.79999998, 13619471.522419333, 1.121554639442056)

In [57]:
results2.shape

NameError: name 'results2' is not defined

In [67]:
results['ABS Error'].sum(), results['Actual'].sum()

(103120848.64505513, 117816568.55)

In [68]:
Sample_description.to_csv('Sample_6597.csv', index=False)  

In [69]:
pd.set_option('display.max_columns', None)
results['ABSCoef'] = abs(results['Coef'])
results = results.sort_values('ABSCoef')
results.head(16)

,Customers,Span,Purchases,Purchase_days,Spend,MAE,MSE,Intercept,Coef,Predicted,Unseasoned,Actual,MAE_pc,Error,ABS Error,Error per day,Target spend per day,Actual per day,MAE per day,Model difference,ABSCoef
5762,54581,0.96,16,13,1175.77,105.20,15637.40,117.51,0.02,1209.04,1177.32,1777.58,0.09,-568.54,568.54,2.27,7.11,4.70,4.21,1.93,0.02
4019,87028,0.92,37,12,384.17,44.43,2808.49,38.28,0.03,384.02,387.29,423.79,0.12,-39.77,39.77,0.16,1.70,1.54,1.78,1.62,0.03
3383,39220,0.94,339,43,5687.60,402.33,200235.23,548.92,-0.04,5182.84,5482.92,686.36,0.07,4496.48,4496.48,17.99,2.75,22.75,16.09,-1.89,0.04
6559,70988,0.97,27,12,606.95,66.73,5630.21,60.90,-0.05,584.83,602.41,4859.15,0.11,-4274.32,4274.32,17.10,19.44,2.43,2.67,-14.43,0.05
99,121536,0.94,20,13,169.46,6.68,70.81,17.21,-0.06,162.61,163.64,127.43,0.04,35.18,35.18,0.14,0.51,0.68,0.27,0.13,0.06
2695,126583,0.99,44,16,348.53,41.23,2765.09,35.22,-0.08,287.75,340.29,347.22,0.12,-59.47,59.47,0.24,1.39,1.39,1.65,1.41,0.08
6989,167578,0.93,21,12,1631.59,119.22,19463.91,163.54,-0.09,1557.48,1623.03,1100.83,0.07,456.65,456.65,1.83,4.40,6.53,4.77,2.94,0.09
3584,81722,0.99,53,18,950.65,108.34,32987.71,94.63,0.10,1012.15,960.32,3004.84,0.11,-1992.69,1992.69,7.97,12.02,3.80,4.33,-3.64,0.10
133,69454,0.92,22,15,1238.64,123.08,19494.11,123.39,0.11,1084.51,1249.28,856.29,0.10,228.22,228.22,0.91,3.43,4.95,4.92,4.01,0.11
3071,87192,0.92,27,18,269.67,25.35,1283.66,27.57,-0.13,264.20,256.17,528.66,0.09,-264.46,264.46,1.06,2.11,1.08,1.01,-0.04,0.13


In [70]:
results.to_csv('LM_season_default.csv', index=False)  